<a href="https://colab.research.google.com/github/walletkun/my-fork-2024-fall-data-science/blob/main/RAG_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 105.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [79]:
import gradio as gr
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.llms import OpenAI
from google.colab import userdata
from langchain.chains import RetrievalQA

In [72]:
qa_chain = None

In [80]:
def process_document(file):
    global qa_chain
    try:
        loader = TextLoader(file.name)
        documents = loader.load()
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        texts = splitter.split_documents(documents)

        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
        vectorstore = Chroma.from_documents(documents=texts, embedding=embeddings)

        llm = OpenAI(temperature=0, api_key=userdata.get('OPENAI_API_KEY'))
        qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())
        return "Document processed successfully!"
    except Exception as e:
        return f"Error processing document: {str(e)}"

In [81]:

def answer_question(question):
    if not qa_chain:
        return "Please upload a document first."
    return qa_chain.run(question)



In [82]:
interface = gr.Interface(
   fn=process_document,
   inputs=gr.File(file_types=['.txt'], type='filepath'),
   outputs="text",
   title="Documentation Assistant"
)

interface2 = gr.Interface(
    fn=answer_question,
    inputs="text",
    outputs="text"
)

demo = gr.TabbedInterface([interface, interface2], ["Upload", "Query"])
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2cd1f913c6c8811723.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
